### Aluno: Matheus Lisboa Oliveira dos Santos
### Matrícula: 0123025804-7M

### Paper
- https://aclanthology.org/D19-5821.pdf

### Code
- https://github.com/patil-suraj/question_generation

- Esse artigo descreve a utilização do modelo de linguagem T5 para a geração de questões. Foi utilizada a base de dados SQuAD para a avaliação. Para a reprodutibilidade utilizaremos os modelo T5 small, por limitações de hardware.

In [ ]:
# !sudo apt-get install python3.7

# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1

# !sudo update-alternatives --config python3

# !sudo apt install python3-pip

# !python --version

# !sudo apt-get install python3.7-distutils

In [12]:
!pip install transformers==3.0.0 nltk nlp==0.2.0
!python -m nltk.downloader punkt
!pip install transformers[torch]
!pip install dill==0.3.5.1
!pip install git+https://github.com/Maluuba/nlg-eval.git@master

/home/matheus/anaconda3/envs/repro/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /home/matheus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  Cloning https://github.com/Maluuba/nlg-eval.git (to revision master) to /tmp/pip-req-build-rc2zprzh
  Running command git clone --filter=blob:none --quiet https://github.com/Maluuba/nlg-eval.git /tmp/pip-req-build-rc2zprzh
  Resolved https://github.com/Maluuba/nlg-eval.git to commit 2ab4528fad5548315cf61e40c2249fec8c8ad233
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 34.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.8 MB/s eta 0:00:00a 0:00:01m
  Preparing metadata (setup.py) ... done
  Created wheel for nlg-ev

### Clonando o repositório com o código base

In [ ]:
!git clone https://github.com/patil-suraj/question_generation

Cloning into 'question_generation'...
remote: Enumerating objects: 268, done.
remote: Total 268 (delta 0), reused 0 (delta 0), pack-reused 268
Receiving objects: 100% (268/268), 299.04 KiB | 1.86 MiB/s, done.
Resolving deltas: 100% (140/140), done.


In [2]:
%cd question_generation

/home/matheus/Documentos/fpcc2-reprodutibilidade/question_generation


### Preparando o conjunto de dados

In [6]:
!python prepare_data.py \
    --task qg \
    --model_type t5 \
    --dataset_path data/squad_multitask/ \
    --qg_format highlight_qg_format \
    --max_source_length 512 \
    --max_target_length 32 \
    --train_file_name train_data_qg_hl_t5.pt \
    --valid_file_name valid_data_qg_hl_t5.pt

07/03/2024 21:39:38 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/t5-spiece.model from cache at /home/matheus/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f
07/03/2024 21:39:38 - INFO - transformers.tokenization_utils -   Adding <sep> to the vocabulary
07/03/2024 21:39:38 - INFO - transformers.tokenization_utils -   Adding <hl> to the vocabulary
07/03/2024 21:39:38 - INFO - nlp.load -   Checking data/squad_multitask/squad_multitask.py for additional imports.
07/03/2024 21:39:38 - INFO - nlp.load -   Found main folder for dataset data/squad_multitask/squad_multitask.py at /home/matheus/anaconda3/envs/repro/lib/python3.7/site-packages/nlp/datasets/squad_multitask
07/03/2024 21:39:38 - INFO - nlp.load -   Found specific version folder for dataset data/squad_multitask/squad_multitask.py at /home/matheus/anaconda3/e

### Treinando o modelo
- Originalmente o autor utiliza o batch size de 32 para o treinamento, porém, por limites de memória de GPU, tivemos que diminuir o batch-size para 28.

In [8]:
%%capture
!python run_qg.py \
    --model_name_or_path t5-small \
    --model_type t5 \
    --tokenizer_name_or_path t5_qg_tokenizer \
    --output_dir t5-small-qg-hl \
    --train_file_path data/train_data_qg_hl_t5.pt \
    --valid_file_path data/valid_data_qg_hl_t5.pt \
    --per_device_train_batch_size 28 \
    --per_device_eval_batch_size 28 \
    --gradient_accumulation_steps 8 \
    --learning_rate 1e-4 \
    --num_train_epochs 10 \
    --seed 42 \
    --do_train \
    --do_eval \
    --logging_steps 100
print("acabou o treinamento")

### Avaliando o modelo

In [11]:
!python eval.py \
    --model_name_or_path /home/matheus/Documentos/fpcc2-reprodutibilidade/question_generation/t5-small-qg-hl \
    --valid_file_path data/valid_data_qg_hl_t5.pt \
    --model_type t5 \
    --num_beams 4 \
    --max_decoding_length 32 \
    --output_path hypothesis_t5-small-qg-hl.txt

100%|█████████████████████████████████████████| 331/331 [04:50<00:00,  1.14it/s]


In [15]:
!nlg-eval --setup /home/matheus/Documentos/fpcc2-reprodutibilidade/

[nltk_data] Downloading package punkt to /home/matheus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Installing to /home/matheus/Documentos/fpcc2-reprodutibilidade/
In case of incomplete downloads, delete the directory and run `nlg-eval --setup /home/matheus/Documentos/fpcc2-reprodutibilidade/' again.

glove2word2vec.py: 100%|█████████████████| 1.00/1.00 [00:00<00:00, 413 chunks/s]
utable.npy:   0%|                             | 0.00/2.23k [00:00<?, ? chunks/s]ERROR:root:Error downloading file, will retry in 60s.
Traceback (most recent call last):
  File "/home/matheus/anaconda3/envs/repro/bin/nlg-eval", line 40, in _download_file
    r.raise_for_status()
  File "/home/matheus/anaconda3/envs/repro/lib/python3.7/site-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://downloads.cs.stanford.edu/nlp/software/stanfor

### Métricas reportadas no artigo
![](imgs/image.png)

Podemos ver que as métricas reportadas pelo autor são muito parecidas. A diferença pode se dar pela alteração no parâmetro de batch_size na etapa de treinamento
  
|         | Autor | Esse Experimento |
|---------|-------|------------------|
| BLEU-4  | 18.5  | 19.0             |
| METEOR  | 24.9  | 25.3             |
| ROUGE-L | 40.1  | 40.8             |

As métricas que obtivemos são superiores as reportadas pelo autor, o que indica que o artigo não sofre com problemas de reprodutibilidade.  

In [17]:
!nlg-eval --hypothesis=hypothesis_t5-small-qg-hl.txt --references=data/references.txt --no-skipthoughts --no-glove 

Using data from /home/matheus/Documentos/fpcc2-reprodutibilidade/
In case of broken downloads, remove the directory and run setup again.
Bleu_1: 0.419301
Bleu_2: 0.310191
Bleu_3: 0.240364
Bleu_4: 0.190926
METEOR: 0.253340
ROUGE_L: 0.408245
CIDEr: 1.805433
Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0,4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - 